In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
from local_utils.retrieval import get_table_data
import pandas as pd

In [4]:
listening_history =  get_table_data("temp_listening_history", "SELECT artist_name, track_name, listened_at FROM temp_listening_history;")

In [5]:
df_history = pd.DataFrame.from_records(listening_history)
df_history = df_history.sort_values("listened_at", ascending=False)
df_history.head()

,artist_name,track_name,listened_at
26947,Oli XL,CONSPIRACY GIRL,2025-12-17 10:03:44+00:00
26948,EsDeeKid,4 Raws,2025-12-17 10:01:17+00:00
26949,The Crying Nudes,joyride,2025-12-17 09:59:40+00:00
26950,Skin On Skin,Let Me Say This,2025-12-17 09:55:27+00:00
26951,Lover's Skit,No Te Metas - Sassy 009 Remix,2025-12-17 09:52:29+00:00


In [6]:
from datetime import datetime, timezone
year = 2025
filter_date = datetime(year, 1, 1, tzinfo=timezone.utc)
df_2025 = df_history[df_history.listened_at >= filter_date]
df_2025.head()

,artist_name,track_name,listened_at
26947,Oli XL,CONSPIRACY GIRL,2025-12-17 10:03:44+00:00
26948,EsDeeKid,4 Raws,2025-12-17 10:01:17+00:00
26949,The Crying Nudes,joyride,2025-12-17 09:59:40+00:00
26950,Skin On Skin,Let Me Say This,2025-12-17 09:55:27+00:00
26951,Lover's Skit,No Te Metas - Sassy 009 Remix,2025-12-17 09:52:29+00:00


## Top artists played

Stacked barplot with top 20 artists. The stacks are their most played songs etc.

In [7]:
df_top_artists = df_2025.value_counts("artist_name").iloc[:20]
df_top_artists.head()

artist_name
Bladee          393
Dean Blunt      359
Vegyn           288
Astrid Sonne    277
SMERZ           274
Name: count, dtype: int64

In [32]:
df_top_artists_2025 = df_2025.query(f"artist_name == {df_top_artists.index.tolist()}")
most_played_artists = df_top_artists_2025[["artist_name", "track_name"]].groupby([ "artist_name", "track_name" ], as_index=False).agg(num_plays=("track_name", "count"))
# most_played_artists["artist_total"] = most_played_artists.groupby("artist_name")["num_plays"].transform("sum")
# most_played_artists = most_played_artists.sort_values(["artist_total", "num_plays"], ascending=[ False, True ])
most_played_artists

,artist_name,track_name,num_plays
0,2hollis,2iss025,1
1,2hollis,all 2s,1
2,2hollis,beginning,1
3,2hollis,burn,2
4,2hollis,cope,2
...,...,...,...
811,bassvictim,Walk Hard (G string edit),1
812,bassvictim,Welcome,9
813,bassvictim,With You,11
814,bassvictim,Wolves Howling,4


In [9]:
import plotly.express as px

fig = px.bar(most_played_artists, y="artist_name", x="num_plays", color="num_plays", hover_name="track_name", color_continuous_scale="blugrn_r")
fig.update_layout(showlegend=False)
fig.update_yaxes(autorange="reversed")

## Top Songs

In [10]:
df_top_songs = df_2025.value_counts("track_name")
df_top_songs

track_name
5                 50
flash             48
bluey vuitton     44
Poplife           43
Woman Lake        43
                  ..
LA to BCN          1
LICK               1
LIEK "S__T"        1
LIGHT'N UP         1
Mildred Pierce     1
Name: count, Length: 4867, dtype: int64

In [11]:
df_top_tracks_2025 = df_2025.query(f"track_name == {df_top_songs.index.tolist()}")
most_played_songs = df_top_tracks_2025[["artist_name", "track_name"]].groupby([ "artist_name", "track_name" ], as_index=False).agg(num_plays=("track_name", "count"))
most_played_songs = most_played_songs.sort_values(["num_plays"], ascending=False).reset_index(drop=True)
most_played_songs["y_labels"] = most_played_songs["artist_name"] + " | " + most_played_songs["track_name"]
most_played_songs

,artist_name,track_name,num_plays,y_labels
0,Dean Blunt,5,50,Dean Blunt | 5
1,2hollis,flash,48,2hollis | flash
2,Babyfather,bluey vuitton,44,Babyfather | bluey vuitton
3,umru,Poplife,43,umru | Poplife
4,Snuggle,Woman Lake,43,Snuggle | Woman Lake
...,...,...,...,...
4978,Disclosure,Omen - Radio Edit,1,Disclosure | Omen - Radio Edit
4979,New York,you,1,New York | you
4980,Niamh,The Golden Ratio,1,Niamh | The Golden Ratio
4981,Dire Straits,Sultans of Swing,1,Dire Straits | Sultans of Swing


In [12]:
fig_songs = px.bar(most_played_songs.iloc[:20], x="num_plays", y="y_labels", color="num_plays", hover_name="track_name", color_continuous_scale="blugrn_r")
fig_songs.update_layout(showlegend=False)
fig_songs.update_yaxes(autorange="reversed")
fig_songs

## Listenership by month

In [13]:
df_2025

,artist_name,track_name,listened_at
26947,Oli XL,CONSPIRACY GIRL,2025-12-17 10:03:44+00:00
26948,EsDeeKid,4 Raws,2025-12-17 10:01:17+00:00
26949,The Crying Nudes,joyride,2025-12-17 09:59:40+00:00
26950,Skin On Skin,Let Me Say This,2025-12-17 09:55:27+00:00
26951,Lover's Skit,No Te Metas - Sassy 009 Remix,2025-12-17 09:52:29+00:00
...,...,...,...
74699,Smino,Hoe-nouns (feat. Thundercat & reggie),2025-01-01 17:58:08+00:00
74700,underscores,Stupid (Can’t run from the urge),2025-01-01 01:17:56+00:00
74701,Snow Strippers,We Both Suffocate,2025-01-01 01:15:13+00:00
74702,Mutilator,Can You Feel It?!,2025-01-01 01:10:27+00:00


In [14]:
df_monthly_dist = df_2025.groupby(df_2025.listened_at.dt.strftime('%Y-%m'), as_index=False).size().sort_values("listened_at")
df_monthly_dist

,listened_at,size
0,2025-01,2186
1,2025-03,1910
2,2025-04,2057
3,2025-05,2567
4,2025-06,1594
5,2025-07,1067
6,2025-08,1334
7,2025-09,1118
8,2025-10,1094
9,2025-11,1129


In [15]:
fig_monthly_dist = px.bar(df_monthly_dist, x="listened_at", y="size")
fig_monthly_dist.show()

In [16]:
df_2025.sort_values("listened_at")

,artist_name,track_name,listened_at
74703,Brutalismus 3000,Europaträume,2025-01-01 01:07:47+00:00
74702,Mutilator,Can You Feel It?!,2025-01-01 01:10:27+00:00
74701,Snow Strippers,We Both Suffocate,2025-01-01 01:15:13+00:00
74700,underscores,Stupid (Can’t run from the urge),2025-01-01 01:17:56+00:00
74699,Smino,Hoe-nouns (feat. Thundercat & reggie),2025-01-01 17:58:08+00:00
...,...,...,...
26951,Lover's Skit,No Te Metas - Sassy 009 Remix,2025-12-17 09:52:29+00:00
26950,Skin On Skin,Let Me Say This,2025-12-17 09:55:27+00:00
26949,The Crying Nudes,joyride,2025-12-17 09:59:40+00:00
26948,EsDeeKid,4 Raws,2025-12-17 10:01:17+00:00


In [17]:
df_history.groupby(df_history.listened_at.dt.strftime('%Y-%m'), as_index=False).size().sort_values("listened_at")

,listened_at,size
0,2021-05,715
1,2021-06,1210
2,2021-07,1307
3,2021-08,1167
4,2021-09,2137
5,2021-10,2428
6,2021-11,2536
7,2021-12,1987
8,2022-01,1974
9,2022-02,2075


### Getting the missing data in the period 2025-01-30 to 2025-03-09

In [18]:
import json

with open("../data/spotify/official_export/Streaming_History_Audio_2024_21.json") as in_file_1:
    missing_pt_1 = json.load(in_file_1)
with open("../data/spotify/official_export/Streaming_History_Audio_2024-2025_22.json") as in_file_2:
    missing_pt_2 = json.load(in_file_2)
with open("../data/spotify/official_export/Streaming_History_Audio_2025_23.json") as in_file_3:
    missing_pt_3 = json.load(in_file_3)

In [19]:
df_missing = pd.DataFrame.from_records(missing_pt_1+missing_pt_2+missing_pt_3)

# filtering only relevant columns
df_missing = df_missing[['ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'spotify_track_uri']]

# convert to datetime
df_missing["ts"] = pd.to_datetime(df_missing["ts"], format="%Y-%m-%dT%H:%M:%SZ", utc=True)

# removing podcast entries
df_missing = df_missing[df_missing.spotify_track_uri.notna()]

# filter the relevant area of data
df_missing = df_missing[df_missing.ts >= datetime(2025, 1, 30, tzinfo=timezone.utc)]
df_missing = df_missing[df_missing.ts <= datetime(2025, 3, 10, tzinfo=timezone.utc)]

df_missing["track_id"] = df_missing.spotify_track_uri.apply(lambda x: x.split(":")[-1])
df_missing

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,spotify_track_uri,track_id
28242,2025-01-30 05:49:30+00:00,185701,Gate 13,Karashnikov,spotify:track:7mXxHnBa1MtQrS1oxFZYy3,7mXxHnBa1MtQrS1oxFZYy3
28243,2025-01-30 05:50:57+00:00,84279,Dont Believe the Hype,Buchecha,spotify:track:0vNqsKGdZ5zi0tDZ2FHkbA,0vNqsKGdZ5zi0tDZ2FHkbA
28244,2025-01-30 05:52:16+00:00,49250,Paro Hour,Luciid,spotify:track:2nMMR3I4OkCx2bXSEenuHB,2nMMR3I4OkCx2bXSEenuHB
28245,2025-01-30 05:54:34+00:00,139361,SPIJKERBOM,Vieze Asbak,spotify:track:3vK4A2BPFWDe4FatXwQhdN,3vK4A2BPFWDe4FatXwQhdN
28246,2025-01-30 05:58:51+00:00,256586,Herz Am Rasen,Johannes Schuster,spotify:track:3Xex3pAdv3t2fFtc1rimRE,3Xex3pAdv3t2fFtc1rimRE
...,...,...,...,...,...,...
35077,2025-03-09 19:34:11+00:00,6861,133 (Ode to Freshman),Chuck Sutton,spotify:track:0Gfg7xE8RfmCXdtCAGVVUD,0Gfg7xE8RfmCXdtCAGVVUD
35078,2025-03-09 19:34:17+00:00,5700,Chess Bling,Chuck Sutton,spotify:track:1UnTjDAmlJafzCKR33m35q,1UnTjDAmlJafzCKR33m35q
35079,2025-03-09 19:34:30+00:00,13165,Rosetta,Chuck Sutton,spotify:track:0UyMWzjPTnEaEEIkDtKKBW,0UyMWzjPTnEaEEIkDtKKBW
35080,2025-03-09 19:34:39+00:00,8277,Breeze,Marius,spotify:track:4Exwb3OMn3WLkAPMwb0BCa,4Exwb3OMn3WLkAPMwb0BCa


In [20]:
# unique_track_ids = [ id.split(":")[-1] for id in df_missing.spotify_track_uri.dropna().unique().tolist() ]
# print(len(unique_track_ids))

# # Get duration of each track to determine if a "scrobble", per last.fm, has happened.
# import json
# import spotipy
# from spotipy.oauth2 import SpotifyOAuth

# with open("../secrets/data-retriever.json") as f:
#     client_json = json.load(f)

# sp = spotipy.Spotify(
#     auth_manager=SpotifyOAuth(
#         client_id=client_json["client_id"],
#         client_secret=client_json["client_secret"],
#         redirect_uri="http://127.0.0.1:8000/callback",
#         ))

# song_duration = []
# for idx in range(0, len(unique_track_ids), 50):
#     track_ids = unique_track_ids[idx:idx+50]

#     results = sp.tracks(track_ids)
#     for track in results["tracks"]:
#         song = {}
#         song["track_id"] = track["id"]
#         song["duration_ms"] = track["duration_ms"]
#         song_duration.append(song)

By retrieving the length of each track, we can apply conditions to narrow down if a track was "scrobbled" according to "last.fm":
- The track must be longer than 30 seconds.
- And the track has been played for at least half its duration, or for 4 minutes (whichever occurs earlier.)

In [21]:
# df_song_duration = pd.DataFrame.from_records(song_duration)
# res_df = pd.merge(df_missing, df_song_duration, on="track_id")
# res_df = res_df[res_df.ms_played >= res_df.duration_ms/2]
# res_df = res_df[res_df.duration_ms > 30000]
# res_df.to_csv("../data/spotify/missing_data_feb_mar_2025.csv")

In [22]:
res_df = pd.read_csv("../data/spotify/missing_data_feb_mar_2025.csv")
res_df = res_df[['ts', 'master_metadata_track_name', 'master_metadata_album_artist_name']]
res_df = res_df.rename(
    {
        "ts": "listened_at", 
        "master_metadata_track_name": "track_name",
        "master_metadata_album_artist_name": "artist_name"
    }, axis=1
)
res_df["listened_at"] = pd.to_datetime(res_df.listened_at, utc=True)
res_df

,listened_at,track_name,artist_name
0,2025-01-30 05:49:30+00:00,Gate 13,Karashnikov
1,2025-01-30 05:54:34+00:00,SPIJKERBOM,Vieze Asbak
2,2025-01-30 05:58:51+00:00,Herz Am Rasen,Johannes Schuster
3,2025-01-30 06:43:48+00:00,Johnny johnny johnny,underscores
4,2025-01-30 06:47:12+00:00,Loansharks,underscores
...,...,...,...
2764,2025-03-09 19:15:27+00:00,Dancing with your eyes closed,Jane Remover
2765,2025-03-09 19:17:21+00:00,source experience analyzed love,meat computer
2766,2025-03-09 19:20:05+00:00,With you,Bassvictim
2767,2025-03-09 19:24:08+00:00,Garden,Maria Somerville


Next steps:
- Integrate the missing data into the result table
- Remove the data from the missing area
- Ensure that the artists from the "missing" dataset matches with the rest of the main table
- Insert into PostgreSQL?

In [28]:
start = datetime(2025, 1, 30, tzinfo=timezone.utc)
end = datetime(2025, 3, 10, tzinfo=timezone.utc)
mask = (df_2025.listened_at<start) & (df_2025.listened_at<end)
df_2025.loc[~mask]

,artist_name,track_name,listened_at
26947,Oli XL,CONSPIRACY GIRL,2025-12-17 10:03:44+00:00
26948,EsDeeKid,4 Raws,2025-12-17 10:01:17+00:00
26949,The Crying Nudes,joyride,2025-12-17 09:59:40+00:00
26950,Skin On Skin,Let Me Say This,2025-12-17 09:55:27+00:00
26951,Lover's Skit,No Te Metas - Sassy 009 Remix,2025-12-17 09:52:29+00:00
...,...,...,...
43660,underscores,Johnny johnny johnny,2025-01-30 06:39:44+00:00
43661,STINGER BROTHERS X U H,НЕ СМЕЙСЯ МНЕ В СПИНУ - HERMANN GEWALT Remix,2025-01-30 06:38:17+00:00
43662,Johannes Schuster,Herz Am Rasen,2025-01-30 05:54:34+00:00
43663,Vieze Asbak,SPIJKERBOM,2025-01-30 05:52:16+00:00


In [24]:
df_2025_full = pd.concat([res_df, df_2025]).sort_values("listened_at").reset_index()
df_2025_full.shape

(19487, 4)

In [25]:
df_dist = df_2025_full.groupby(df_2025_full.listened_at.dt.strftime('%Y-%m'), as_index=False).size().sort_values("listened_at")
fig_dist = px.bar(df_dist, x="listened_at", y="size")
fig_dist.show()

In [31]:
df_2025_full.groupby(df_2025_full.listened_at.dt.strftime('%Y-%m'), as_index=False).size().sort_values("listened_at")

,listened_at,size
0,2025-01,2427
1,2025-02,1891
2,2025-03,2547
3,2025-04,2057
4,2025-05,2567
5,2025-06,1594
6,2025-07,1067
7,2025-08,1334
8,2025-09,1118
9,2025-10,1094


need data setup function